In [1]:
#%load_ext autoreload
#%autoreload 2
import numpy as np
from vvasp.VizClasses import NeuropixelsChronicHolder
from vvasp.atlas_utils import Atlas
import pyvista as pv
from labdata.schema import *

[2024-05-16 18:43:14,998][INFO]: Connecting mmelin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306
[2024-05-16 18:43:15,398][INFO]: Connected mmelin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306


In [29]:
from labdata.schema import *
from labdata import chronic_paper as cp
cp.TargetedRegion()


subject_name unique mouse id,procedure_type Defines procedures that are not an experimental session,procedure_datetime,probe_id probe id to keep track or re-uses,atlas_id,region_acronym,holder_id,hemisphere
MM008,chronic implant,2023-03-14 20:00:00,19454421152,allen-ccf,MOp,0,left
MM008,chronic implant,2023-03-14 20:00:00,20097902741,allen-ccf,ACA,0,left
MM008,chronic implant,2023-03-14 20:00:00,20097902851,allen-ccf,VISp,0,left


In [26]:
subjects = cp.IncludedSubjects().fetch('subject_name')
subjects
subject = 'MM008'
targets = [['ACA','CP'], ['MOp','CP'], ['VISp']]
holder_ids = [0,0,0]
hemisphere = 'left'
insertions = (ProbeInsertion() & dict(subject_name=subject)).proj().fetch(order_by='insertion_ap',as_dict=True)[::-1]

for h, ins, targs in zip(holder_ids, insertions, targets):
    for t in targs:
        print(t)
        reg = (Atlas.Region() & dict(region_acronym=t)).proj().fetch1()
        key = dict(**ins, **reg, holder_id=h, hemisphere=hemisphere)
        print(key)
        cp.TargetedRegion.insert1(key, skip_duplicates=True)

ACA
{'subject_name': 'MM008', 'procedure_type': 'chronic implant', 'procedure_datetime': datetime.datetime(2023, 3, 14, 20, 0), 'probe_id': '20097902741', 'atlas_id': 'allen-ccf', 'region_acronym': 'ACA', 'holder_id': 0, 'hemisphere': 'left'}
CP
{'subject_name': 'MM008', 'procedure_type': 'chronic implant', 'procedure_datetime': datetime.datetime(2023, 3, 14, 20, 0), 'probe_id': '20097902741', 'atlas_id': 'allen-ccf', 'region_acronym': 'CP', 'holder_id': 0, 'hemisphere': 'left'}
MOp
{'subject_name': 'MM008', 'procedure_type': 'chronic implant', 'procedure_datetime': datetime.datetime(2023, 3, 14, 20, 0), 'probe_id': '19454421152', 'atlas_id': 'allen-ccf', 'region_acronym': 'MOp', 'holder_id': 0, 'hemisphere': 'left'}
CP
{'subject_name': 'MM008', 'procedure_type': 'chronic implant', 'procedure_datetime': datetime.datetime(2023, 3, 14, 20, 0), 'probe_id': '19454421152', 'atlas_id': 'allen-ccf', 'region_acronym': 'CP', 'holder_id': 0, 'hemisphere': 'left'}
VISp
{'subject_name': 'MM008', '

In [ ]:
for dict in insertions:
    print(dict)

In [ ]:
insertions2plot = Probe() * ProbeInsertion() & f'subject_name = "{SUBJECT}"'
#insertions2plot = Probe() * ProbeInsertion() & 'subject_name = "JC131"'
#insertions2plot = Probe() * ProbeInsertion() & 'subject_name = "JC164"'
insertions = insertions2plot.fetch(format='frame')
insertions.head()

In [ ]:
plotter = pv.Plotter(notebook=True)
plotter.show()
#from pyvistaqt import BackgroundPlotter
#plotter = BackgroundPlotter()

atlas = Atlas(plotter, min_tree_depth=8, max_tree_depth=8)
#atlas.add_atlas_region_mesh('MOp')
atlas.add_atlas_region_mesh('LP')
#atlas.add_atlas_region_mesh('ACAd')
#atlas.add_atlas_region_mesh('ACAv')
atlas.add_atlas_region_mesh('VISp')
#atlas.add_atlas_region_mesh('CP')

probes = []
for i, entrypoint in insertions.iterrows():
    entry_point = entrypoint[['insertion_ml','insertion_ap']].values
    depth = entrypoint['insertion_depth']
    angles = entrypoint[['insertion_el','insertion_spin','insertion_az']].values
    if entrypoint.probe_type == '24':
        prb = NeuropixelsChronicHolder('NP24','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    elif entrypoint.probe_type == '0':
        prb = NeuropixelsChronicHolder('NP1','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    prb.drive_probe_from_entry(entry_point, angles, depth)
    prb.make_inactive()
    #prb.set_location(np.concatenate([entry_point,np.array([0])]), angles*0)
    probes.append(prb)

plotter.isometric_view()

In [ ]:
from pathlib import Path
savedir = Path(r'C:\Data\churchland\chronic_manuscript_figs\raw_plots\trajectories')

#plotter.save_graphic(savedir / f'{SUBJECT}_trajectories.pdf', raster=False)
plotter.screenshot(savedir / f'{SUBJECT}_trajectories.png', return_img=False, scale=5)